# Q2 Using ADQL to Search for Bright Stars Around the Open Cluster M67

A colleague is interested in the open cluster Messier 67 (RA = 132.825 deg, Dec = 11.8 deg)
and is considering an observation proposal using the 2dF fibre positioner and HERMES
spectrograph (effective for Gaia G band magnitudes < 14). They need to know if there
are enough bright stars in this region for observation. Your task is to assist by querying
data from Gaia DR3 and performing some basic analysis.

- #### Download all stars within 1 degree of the center of Messier 67 that are brighter than G = 14 in Gaia DR3 (the table is called gaiadr gaia source) and include a crossmatch these stars with the 2MASS catalog and report your ADQL query text (use the """query""" notation)

- #### Determine how many stars are returned from the initial query.a

- #### Identify any stars with bad 2MASS photometry, where ph qual is not ’AAA’.

- #### Identify any stars with negative (or non-positive) parallaxes in the Gaia data.

- #### Apply these two quality cuts (removing stars with bad 2MASS photometry and non-positive parallaxes). After applying the cuts, determine how many stars remain.

- #### Using the remaining stars, generate a figure with two panels (1 point per panel):


(a) A color-magnitude diagram (CMD) of Gaia BP-RP vs. absolute G magnitude.


(b) A 2MASS J-Ks vs. apparent K magnitude diagram.

- #### Save the figure as figures/cmds M67.png with a resolution of 200 dots per inch

- #### Give your colleague a recommendation for the potential proposal when only judging the fibre usage.

# Question 3 The radial metallicity relation in simulated data 

The radial metallicity relation is a function that describes the change of metallicity -
here the gas phase metallicity $A(O) = log10(NO/NH) + 12$ - along the galactocentric
radius $R_{Gal}$.. Understanding the radial metallicity gradient in galaxies provides critical
insights into their formation and evolutionary processes, such as inside-out formation,
gas accretion, outflows, and radial migration. A lot of work has been done through
observational studies (e.g. Ho et al., 2017, ApJ, 846, 39) and a few simulations (e.g.
Grand et al., 2016, MNRAS, 460, 94), but more works needs to be done to understand
the radial metallicity gradient!
Your colleague has just finished an exciting cosmological simulation that traces the
gas phase metallicity for a Milky Way analogue, that is, a spiral galaxy. They have
limited the simulation data to the positions (x, y, z) of the innermost gas particles
(RGal. < 25 kpc) and their gas phase metallicity A(O) and uploaded them as a FITS
file for you here: https://github.com/svenbuder/astr4004_2024_week7/blob/main/data/nihao_uhd_simulation_g8.26e11_xyz_positions_and_oxygen_ao.fits

#### - Download the file from the link above into **data/**. Load the file with python and then perform the following tasks to create figures that are saved in **figures/**:

(a) Logarithmic density plot of RGal. vs. A(O), with a linear fit and legend.

(b) Residuals of the fit, $R_{Gal}$. vs. $\Delta A(O)$.

#### Use a python fitting tool to fit a linear function to the data, reporting the intercept and slope with uncertainties. Include any hyperparameters used.

#### Discuss where the linear model fits well and where it does not. Use statistical metrics, such as the root mean squares or other goodness-of-fit indicators, to quantify the performance of your linear fit in general and regions with larger residuals.

#### Plot a 3-panel figure for the x vs. y plane using the same bins and sensible colormaps:

(a) 2D-histogram of the median simulated A(O)

(b) 2D-histogram of the median fitted A(O)

(c) 2D-histogram of the median residuals ∆A(O)

#### Describe your choice of 2D bins. Discuss what details would be missed with fewer bins or problems encountered with more bins.

#### Analyze the residuals in more detail and propose an explanation for any patterns you observe.